In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
import util
%autosave 5

Autosaving every 5 seconds


Autosaving every 5 seconds


In [ ]:
possibleConvFilters1 = util.createRangeFromMidpoint(13,26)
possibleConvFilters2 = util.createRangeFromMidpoint(46,92)
possibleConvFilters3 = util.createRangeFromMidpoint(17,34)
possibleConvFilters4 = util.createRangeFromMidpoint(16,32)

possibleNumberOfFCLayers = util.createRangeFromMidpoint(27,54)
possibleNumberOfNeuronsPerFCLayer = util.createRangeFromMidpoint(33,66)

possibleNumberOfEpochs = util.createRangeFromMidpoint(22,44)
dropoutCriticalPoints = (0,1)
adamLearningRateCriticalPoints = (1e-4,1e-2)
L2CriticalPoints = (1e-2,1e3)   

trial = 0

train = util.loadData("training")
dev,test = util.loadData("testing",.5)

choose = np.random.choice

n_convFilters1 = []
n_convFilters2 = []
n_convFilters3 = []
n_convFilters4 = []
n_FCLayers = []
n_NeuronsPerFCLayers = []
n_Epochs = []
dropoutRates = []
adamLearningRates = []
L2Rates = []
trainScores = []
devScores = []

while trial < 50:
    convFilters1 = choose(possibleConvFilters1)
    convFilters2 = choose(possibleConvFilters2)
    convFilters3 = choose(possibleConvFilters3)
    convFilters4 = choose(possibleConvFilters4)

    numberOfFCLayers = choose(possibleNumberOfFCLayers)
    numberOfNeuronsPerFCLayer = choose(possibleNumberOfNeuronsPerFCLayer)

    numberOfEpochs = choose(possibleNumberOfEpochs)
    
    dropoutRate = util.generateDropoutRate(dropoutCriticalPoints[0],dropoutCriticalPoints[1])
    adamLearningRate = util.generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
    L2Rate = util.generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
    

    model = util.createModel(convFilters1, convFilters2, convFilters3, convFilters4,
                        numberOfFCLayers, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate,L2Rate)

    model.fit(train,epochs=numberOfEpochs,verbose=0)

    model_path = f'../Models/FlipDetectionModelTrials/fd_model_{trial}.h5'
    model.save(model_path)
    model_size = os.path.getsize(model_path) / (1024 * 1024)
    if model_size < 40:
        print()
        print('trainScore')
        trainScore = model.evaluate(train)[1]
        print('devScore')
        devScore = model.evaluate(dev)[1]

        if devScore > 0.91:
            model_path = f'../Models/fd_model_{trial}.h5'
            model.save(model_path)

        n_convFilters1.append(convFilters1)
        n_convFilters2.append(convFilters2)
        n_convFilters3.append(convFilters3)
        n_convFilters4.append(convFilters4)
        
        n_FCLayers.append(numberOfFCLayers)
        n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
        n_Epochs.append(numberOfEpochs)
        
        adamLearningRates.append(adamLearningRate)
        dropoutRates.append(dropoutRate)
        L2Rates.append(L2Rate)
        trainScores.append(trainScore)
        devScores.append(devScore)
        
        print('concluding trial ',trial)
        trial += 1
    else:
        print(f'redoing trial {trial}. Model was {model_size}MB.')
        failedTrial = util.createModelParametersDF([convFilters1],[convFilters2],[convFilters3],[convFilters4],
                                              [numberOfFCLayers],[numberOfNeuronsPerFCLayer],[numberOfEpochs],
                                              [dropoutRate],[adamLearningRate],[L2Rate],[np.nan],[np.nan])
        display(failedTrial)
    
        
    if (trial % 10 == 9): 
        modelParametersDF = util.createModelParametersDF(n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,
                                                n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,
                                                dropoutRates,adamLearningRates,L2Rates,trainScores,devScores)
        modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
        display(modelParametersDF)
        
        top5 = modelParametersDF[0:5]
        possibleConvFilters1 = util.createRangeFromMidpoint(np.mean(top5['n_convFilters1']),6*np.std(top5['n_convFilters1']))
        possibleConvFilters2 = util.createRangeFromMidpoint(np.mean(top5['n_convFilters2']),6*np.std(top5['n_convFilters2']))
        possibleConvFilters3 = util.createRangeFromMidpoint(np.mean(top5['n_convFilters3']),6*np.std(top5['n_convFilters3']))
        possibleConvFilters4 = util.createRangeFromMidpoint(np.mean(top5['n_convFilters4']),6*np.std(top5['n_convFilters4']))
    
        possibleNumberOfFCLayers = util.createRangeFromMidpoint(np.mean(top5['n_FCLayers']),6*np.std(top5['n_FCLayers']))
        possibleNumberOfNeuronsPerFCLayer = util.createRangeFromMidpoint(np.mean(top5['n_NeuronsPerFCLayers']),6*np.std(top5['n_NeuronsPerFCLayers']))
    
        possibleNumberOfEpochs = util.createRangeFromMidpoint(np.mean(top5['n_Epochs']),6*np.std(top5['n_Epochs']))
        dropoutCriticalPoints = util.calculateCriticalPoints(top5['dropoutRate'])
        adamLearningRateCriticalPoints = util.calculateLogisticCriticalPoints(top5['adamLearningRates'])
        L2CriticalPoints = util.calculateLogisticCriticalPoints(top5['L2Rates'])

        n_convFilters1 = []
        n_convFilters2 = []
        n_convFilters3 = []
        n_convFilters4 = []
        n_FCLayers = []
        n_NeuronsPerFCLayers = []
        n_Epochs = []
        dropoutRates = []
        adamLearningRates = []
        L2Rates = []
        trainScores = []
        devScores = []
            
    

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-07-07 23:15:06.351111: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-07-07 23:15:06.351257: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-07-07 23:15:07.125457: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz



trainScore
75/75 [==============================] - 14s 176ms/step - loss: 0.3496 - f1_score: 0.6503
devScore


2024-07-08 00:01:48.311741: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]
2024-07-08 00:01:48.311903: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [299]
	 [[{{node Placeholder/_0}}]]


10/10 [==============================] - 2s 133ms/step - loss: 0.3657 - f1_score: 0.6277
concluding trial  0

trainScore
75/75 [==============================] - 13s 161ms/step - loss: 0.3520 - f1_score: 0.6481
devScore
10/10 [==============================] - 2s 127ms/step - loss: 0.3692 - f1_score: 0.6244
concluding trial  1

trainScore
75/75 [==============================] - 16s 205ms/step - loss: 0.3525 - f1_score: 0.6477
devScore
10/10 [==============================] - 2s 153ms/step - loss: 0.3667 - f1_score: 0.6384
concluding trial  2

trainScore
75/75 [==============================] - 17s 217ms/step - loss: 0.3520 - f1_score: 0.6477
devScore
10/10 [==============================] - 2s 170ms/step - loss: 0.3743 - f1_score: 0.6196
concluding trial  3

trainScore
75/75 [==============================] - 9s 108ms/step - loss: 4.0957 - f1_score: 0.6500
devScore
10/10 [==============================] - 1s 66ms/step - loss: 4.1130 - f1_score: 0.6320
concluding trial  4

trainScore
7

...

redoing trial 31. Model was 40.14894485473633MB.


,n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
0,11,105,23,60,22,71,50,0.702978,0.000322,1.039386e+06,NaN,NaN


redoing trial 31. Model was 85.6467056274414MB.


,n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
0,21,129,37,89,39,100,25,0.686858,0.345447,4.727073e-12,NaN,NaN



trainScore
75/75 [==============================] - 29s 381ms/step - loss: 0.3507 - f1_score: 0.6493
devScore
10/10 [==============================] - 4s 316ms/step - loss: 0.3691 - f1_score: 0.6448
concluding trial  31

trainScore
75/75 [==============================] - 23s 303ms/step - loss: 0.3507 - f1_score: 0.6491
devScore
10/10 [==============================] - 3s 248ms/step - loss: 0.3650 - f1_score: 0.6283
concluding trial  32

trainScore
75/75 [==============================] - 30s 388ms/step - loss: 0.3514 - f1_score: 0.6482
devScore
10/10 [==============================] - 4s 340ms/step - loss: 0.3686 - f1_score: 0.6180
concluding trial  33

trainScore
75/75 [==============================] - 20s 259ms/step - loss: 0.3501 - f1_score: 0.6498
devScore
10/10 [==============================] - 3s 211ms/step - loss: 0.3662 - f1_score: 0.6332
concluding trial  34
